# Parsing Instagram Data Structures into a Schema_df

 The purpose of his jupyter notebook is to parse the collected data structures in earlier iterations of utilising the data donation tool into a schema_df which can be used to inform future iterations of the data donation tool.

In [26]:
import pandas as pd
import json
import numpy as np
from pathlib import Path  

In [27]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"

## Creating functions
### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [28]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']

#Define the column names
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']



# Define the function 'process_col_path()'
def process_col_path(row, columns, data_types):

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """

    row['data_type'] = ''
    for column in columns:

        #If the value stored in the column is found in the list 'data_types', 
        if row[column] in data_types:
            # this value is placed in the column 'data_type'
            row['data_type'] = row[column]
            # and the original value is replaced with NA
            row[column] = np.nan

        #If the value is not found in the 'data_types' list, the original value is returned
        else:
            row[column]
    return row


### file_paths()
The 'file_paths()' function splits up the paths to where the JSON file is stored in the folder and provides the name of the json file. 

In [29]:
def file_paths(df):
    
    df['value'] = df['value'].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

### string_to_dict()
As the JSON files are loaded as strings, they need to be converted to dictionaries to extract the values and be cleaned. 

In [30]:
def string_to_dict(s):
    # Check if the items needed for splitting are present. If not present it does not need to be splitted and the orginal value is returned
    if ',' not in s and ':' not in s:
        return s
    
    # Create an empty dictionary
    result = {}

    # Split the items by comma (split into key-value pairs)
    items = s.split(',')

    # For each item in the original dictionary
    for item in items:
        # Check if it contains a key-value pair, if not continue
        if ':' not in item:
            continue  

        # Split the key-value pair into a variable 'key' and a variable 'value'
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issue

        # Try to strip any white spaces from the keys and values 
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        # If not possible, continue
        except Exception as e:
            continue
        
        # Save the converted and cleaned dictionary
        result[key] = value
    return result
    

### detect_list()

Due to lists in unexpected places in the JSON files and lengthy json paths, we need to identify the positions of lists to later select the correct get() function

In [31]:
# Assign list if the data in the original structure is a list
    
def detect_list(x):
    # If the data type of the value is list, 'LIST' is assigned in the '_LIST' columns (see'column_paths())
    if isinstance(x, list):
        return 'LIST'
    # If the value is missing, 'MISSING' is assigned
    elif pd.isna(x):
        return 'MISSING'
    # If the value is 'No data' (there is an empty place holder) 'MISSING' is assigned
    elif x == 'No data':
            return 'MISSING'
    # if the value is not missing and is not a list, 'NO LIST' is assigned
    else:
        return 'NO LIST'

### columns_paths()
Through this function, we map the json paths to each value by putting the keys in separate columns: colpath_{1,2,3,4,5} (see 'row_column_paths()')

In [32]:
def column_paths(df, col_var, get_var, unlist_var, list_var):

    # Convert to dict if it's a string
    df[col_var] = df[col_var].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
        # Extract nested values using keys
    """
    1. Selects the key obtained in the previous iteration stored in 'get_var'
    2. The key put into a get() function which extracts values of a dictionary based on the keys
    3. The get() function extracts the values from the (nested) dictionary stored in 'col_var'
    4. If the get() function fails, None is returned
    5. The steps above are only performed if the value stored in 'col_var' is a dictionary, otherwise None is returned
    6. All steps above are executed for each row in the df
    """

    df[get_var] = df.apply(
    lambda row: list(row[col_var].keys()) if isinstance(row[col_var], dict) else row[col_var], axis=1)

    
    df = df.explode(get_var)

    
    # Extract nested values using keys
    df[unlist_var] = df.apply(lambda row: row[col_var].get(row[get_var], None) if isinstance (row[col_var], dict) else None, axis=1)

    # For each each column_path, check if the value contains a list
    df[f'{list_var}_LIST'] = df[unlist_var].apply(detect_list)
    
     # Unlist data to avoid double nested lists 
    df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)



    return df

### row_column_paths()
This function prepares the row paths and consequently executes the column_paths function to obtain the paths of keys (stored in individual columns) to reach the lowest value which is where the actual data is stored


In [33]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = df['variable'] 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    
    # Initiate the list of columns needed to inform the function column_paths()
    col_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    list_var = ['col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    # Execute the colums_path() function
    # zip is necessary due to the large number of variables needed in the column_paths function
    for r, g, u, l in zip(col_var, get_var, unlist_var, list_var):
        df = column_paths(df, r, g, u, l)
        
    return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [34]:
def clean_and_store(df, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'row_path', 
                    'col_path_1', 
                     'col_path_2', 'col_path_2_LIST',
                     'col_path_3',  'col_path_3_LIST',
                     'col_path_4', 'col_path_4_LIST',
                     'col_path_5', 'col_path_5_LIST',
                     'data_type', 'col_path_1_values',
                     'col_path_2_values','col_path_3_values',
                     'col_path_4_values']]
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # fill na values with 'Missing'
    df = df.fillna('Missing')


    # Save the DataFrame 
    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [35]:
def structure_donations(data):

    """
    data: The unprocessed data structure JSON that will be processed
    """

    # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    # Load JSON file (data structures)
    with open(data, 'r') as f:
        data = json.load(f)
    
    # Strip top-level key
    data = list(data.values())[0]


    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    # Execute the 'file_paths()' function and store the result in df
    df = file_paths(df)

    # Execute the 'row_column_paths()' function and store the result in df
    df = row_column_paths(df)
    
    # Execute the 'process_col_path()' function and store the result in df
    df = df.apply(lambda row: process_col_path(row, columns, data_types), axis=1)

    # Execute the 'clean_and_store()' function and store the result in df
    df = clean_and_store(df, file_name)
    
    
    return df


## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [36]:
input_directory = Path(f'{main_path}TikTok/Input_test')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input_test


In [37]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        structure_donations(file)
        

## Merge all data structures into one schema_df

In [38]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()

# Replace 'Missing' with NaN
merged_df = merged_df.replace('Missing', np.nan)


# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)

/tmp/ipykernel_85756/766349402.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.replace('Missing', np.nan)
